# 📓 Procesing La.data files into County Level Unemployment Statistics 

This notebook processes La.data files to build a set of county-level unemployment statistics, per state. It creates a set of CSV files, for use in other data analysis. 

**Place the la.data files, including la_area and la_series into the same directory as this folder.**

They can be found at either https://download.bls.gov/pub/time.series/la/ or at this google drive link : https://drive.google.com/drive/folders/1rPVdtOSz7vInRqKBDUe-VXzEK2rjw7G1?usp=share_link

You will need a valid Umich Email to access the Drive link. 

In [8]:
# --- 1. Config ---
# Manually defined list of dictionary entries for each FIPS identifier and la.data file. 

STATE_CONFIGS = [
    {"fips": "01", "file": "la.data.7.Alabama"},
    {"fips": "02", "file": "la.data.8.Alaska"},
    {"fips": "04", "file": "la.data.9.Arizona"},
    {"fips": "05", "file": "la.data.10.Arkansas"},
    {"fips": "06", "file": "la.data.11.California"},
    {"fips": "08", "file": "la.data.12.Colorado"},
    {"fips": "09", "file": "la.data.13.Connecticut"},
    {"fips": "10", "file": "la.data.14.Delaware"},
    {"fips": "11", "file": "la.data.15.DC"},
    {"fips": "12", "file": "la.data.16.Florida"},
    {"fips": "13", "file": "la.data.17.Georgia"},
    {"fips": "15", "file": "la.data.18.Hawaii"},
    {"fips": "16", "file": "la.data.19.Idaho"},
    {"fips": "17", "file": "la.data.20.Illinois"},
    {"fips": "18", "file": "la.data.21.Indiana"},
    {"fips": "19", "file": "la.data.22.Iowa"},
    {"fips": "20", "file": "la.data.23.Kansas"},
    {"fips": "21", "file": "la.data.24.Kentucky"},
    {"fips": "22", "file": "la.data.25.Louisiana"},
    {"fips": "23", "file": "la.data.26.Maine"},
    {"fips": "24", "file": "la.data.27.Maryland"},
    {"fips": "25", "file": "la.data.28.Massachusetts"},
    {"fips": "26", "file": "la.data.29.Michigan"},
    {"fips": "27", "file": "la.data.30.Minnesota"},
    {"fips": "28", "file": "la.data.31.Mississippi"},
    {"fips": "29", "file": "la.data.32.Missouri"},
    {"fips": "30", "file": "la.data.33.Montana"},
    {"fips": "31", "file": "la.data.34.Nebraska"},
    {"fips": "32", "file": "la.data.35.Nevada"},
    {"fips": "33", "file": "la.data.36.NewHampshire"},
    {"fips": "34", "file": "la.data.37.NewJersey"},
    {"fips": "35", "file": "la.data.38.NewMexico"},
    {"fips": "36", "file": "la.data.39.NewYork"},
    {"fips": "37", "file": "la.data.40.NorthCarolina"},
    {"fips": "38", "file": "la.data.41.NorthDakota"},
    {"fips": "39", "file": "la.data.42.Ohio"},
    {"fips": "40", "file": "la.data.43.Oklahoma"},
    {"fips": "41", "file": "la.data.44.Oregon"},
    {"fips": "42", "file": "la.data.45.Pennsylvania"},
    {"fips": "44", "file": "la.data.47.RhodeIsland"},
    {"fips": "45", "file": "la.data.48.SouthCarolina"},
    {"fips": "46", "file": "la.data.49.SouthDakota"},
    {"fips": "47", "file": "la.data.50.Tennessee"},
    {"fips": "48", "file": "la.data.51.Texas"},
    {"fips": "49", "file": "la.data.52.Utah"},
    {"fips": "50", "file": "la.data.53.Vermont"},
    {"fips": "51", "file": "la.data.54.Virginia"},
    {"fips": "53", "file": "la.data.56.Washington"},
    {"fips": "55", "file": "la.data.58.Wisconsin"},
    {"fips": "56", "file": "la.data.59.Wyoming"},
]

In [9]:
# --- 2. Setup Area, Area Type, and Series lookup tables ---
import pandas as pd
import os

# Load lookup files, needed to link data to each state
la_area = pd.read_csv("la.area", sep="\t", dtype=str)

la_series = pd.read_csv(
    "la.series", 
    sep="\t", header=None,
    names=[
        "series_id","area_type_code","area_code",
        "measure_code","seasonal","srd_code",
        "series_title","footnote_codes",
        "begin_year","begin_period",
        "end_year","end_period"
    ],
    dtype=str
)


print("Lookup files loaded.")

Lookup files loaded.


In [10]:
all_datasets = []

output_dir = 'state_csv'
os.makedirs(output_dir, exist_ok=True)

for config in STATE_CONFIGS:


    fips = config["fips"]
    data_file = config["file"]
    state_name = data_file.split(".")[-1]

    
    # Identifies counties by selected state
    state_counties = la_area[
        (la_area["area_type_code"] == "F") &  # F is the State identifier in the La_area file
        (la_area["area_code"].str.startswith("CN" + fips))
    ].copy()
    
    if state_counties.empty:
        print(f"⚠️ No counties found for {data_file}")
        continue
    
    # Load data
    la_data = pd.read_csv(
        data_file,
        sep="\t", header=None,
        names=["series_id","year","period","value","footnote_codes"],
        dtype=str
    )
    
    # Filter series
    state_series = la_series[la_series["area_code"].isin(state_counties["area_code"])].copy()
    
    # Merge
    merged = (
        la_data
        .merge(state_series, on="series_id")
        .merge(state_counties, on="area_code")
    )
    
    if merged.empty:
        print(f"⚠️ Empty merge for {data_file}")
        continue
    
    merged["year"] = merged["year"].astype(int)
    merged["month"] = merged["period"].str[1:].astype(int)
    merged["value"] = pd.to_numeric(merged["value"], errors="coerce")
    
    # Pivot
    pivoted = merged.pivot_table(
        index=["area_text","year","month"],
        columns="measure_code",
        values="value"
    ).reset_index()
    
    pivoted.rename(columns={
        "03": "unemployment_rate",
        "04": "labor_force",
        "05": "employment",
        "06": "unemployment"
    }, inplace=True)
    
    state_final = pivoted.rename(columns={"area_text": "county"})
    all_datasets.append(state_final)
    
    # Save each state separately
    output_path = os.path.join(output_dir, f"{state_name}_county_unemployment.csv")
    state_final.to_csv(output_path, index=False)
    print(f"✅ Saved {data_file} with {len(state_final)} rows as {state_name}")

✅ Saved la.data.7.Alabama with 30552 rows as Alabama
✅ Saved la.data.8.Alaska with 12718 rows as Alaska
✅ Saved la.data.9.Arizona with 6840 rows as Arizona
✅ Saved la.data.10.Arkansas with 34200 rows as Arkansas
✅ Saved la.data.11.California with 26449 rows as California
✅ Saved la.data.12.Colorado with 29184 rows as Colorado
✅ Saved la.data.13.Connecticut with 4104 rows as Connecticut
✅ Saved la.data.14.Delaware with 1368 rows as Delaware
✅ Saved la.data.15.DC with 457 rows as DC
✅ Saved la.data.16.Florida with 30553 rows as Florida
✅ Saved la.data.17.Georgia with 72504 rows as Georgia
✅ Saved la.data.18.Hawaii with 1824 rows as Hawaii
✅ Saved la.data.19.Idaho with 20064 rows as Idaho
✅ Saved la.data.20.Illinois with 46512 rows as Illinois
✅ Saved la.data.21.Indiana with 41952 rows as Indiana
✅ Saved la.data.22.Iowa with 45144 rows as Iowa
✅ Saved la.data.23.Kansas with 47880 rows as Kansas
✅ Saved la.data.24.Kentucky with 54720 rows as Kentucky
✅ Saved la.data.25.Louisiana with 29100

## Summary

With the LA.data files processed successfully, you should have 50 CSVs within a subfolder called state_csv. These correspond to each state, named with the state name from the La.data file.

- In future notebooks, we will process the BRFSS data into the same format using a pivot table and a renaming map constructed from the BRFSS codebook.